# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다.

## Data 

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보 
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("assignment3_creditcard.csv")

In [3]:
data.head()

In [4]:
data.info()

In [5]:
data.describe()

In [6]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1995, test_size = 0.3, stratify = y)

In [7]:
# 성능지표 계산 함수 만들기
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬 confusion matrix')
    print(confusion)
    print('정확도 accuracy: {0:.4f}, 정밀도 precision: {1:.4f}, 재현율 recall: {2:.4f}, F1: {3:.4f}, AUC: {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [8]:
# 로지스틱 회귀분석 진행
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_pred_proba = lr.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, lr_pred, lr_pred_proba)

### 해석 
* 정확도 (Accuracy): 정확도는 0.9977로, 매우 높은 정확도를 갖고 있다고 볼 수 있습니다.
* 정밀도 (Precision): 정밀도는 0.9500으로, Positive로 예측한 것 중 95%가 실제로 Positive라고 할 수 있습니다.
* 재현율 (Recall): 재현율은 0.7703으로, 실제 Positive 중 77%를 모델이 정확하게 예측했다고 해석할 수 있습니다.
* F1 (F1 Score): F1은 0.8507로, 정밀도와 재현율의 균형있는 성능을 보여줍니다.
* AUC: AUC는 0.9712로, 모델의 분류 성능이 좋음을 나타냅니다.

In [9]:
# 반복적으로 모델 학습/예측/평가 함수 생성
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)

In [10]:
# 성능 개선 시도 : 이상치 제거 

# dataframe의 correlation 구한 뒤 heatmap으로 나타내기
plt.figure(figsize=(9, 9))
corr = data.corr()
sns.heatmap(corr, cmap='RdBu')

### Class feature와 음의 상관관계가 가장 높은 feature는 V14, V17

In [11]:
# V14에 대해서 이상치 제거하기 
def get_outlier(data=None,column=None,weight=1.5):
    # fraud에 해당하는 column 데이터만 추출, 1사분위와 3사분위 지점을 구함
    fraud = data[data['Class'] == 1][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    # IQR을 구하고 IQR에 1.5를 곱해 최댓값과 최솟값 지점 구함
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    # 이상치 index 반환
    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index

In [12]:
outlier_index = get_outlier(data=data, column='V14', weight=1.5)
print('이상치 데이터 인덱스: ', outlier_index)

In [13]:
# 추출된 이상치 제거 후 모델 적용
data.drop(outlier_index, axis=0, inplace=True)

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1995, test_size = 0.3, stratify = y)

get_model_train_eval(lr, ftr_train = X_train, ftr_test = X_test, tgt_train = y_train, tgt_test = y_test)